# Modeling
Build, train and tune ML models to predict heart disease using preprocessed data

In [23]:
# Load Libaries
import pandas as pd
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Load the saved data splits
with open("split_data/x_train.pkl", "rb") as f:
    x_train = pickle.load(f)

with open("split_data/y_train.pkl", "rb") as f:
    y_train = pickle.load(f)

with open("split_data/x_test.pkl", "rb") as f:
    x_test = pickle.load(f)

with open("split_data/y_test.pkl", "rb") as f:
    y_test = pickle.load(f)

## Dealing with Class Imbalance

Note that the dataset is imbalanced, where most entries belong to the "Absent" class for heart disease status. It is important to deal with this imbalance as it may cause the model to be bias and predict the majority class more. Oversampling to balance the classes in the training set will allow the model to better learn patterns associated with the minority class.

In [24]:
# Combine x_train and y_train for oversampling
training_data = pd.concat([x_train, y_train], axis = 1)

# Oversampling (for equal majority and minority class)
majority_class = training_data[training_data["heart_disease_status"] == "Absent"]
minority_class = training_data[training_data["heart_disease_status"] == "Present"]
minority_oversampled = minority_class.sample(n = len(majority_class), replace = True, random_state = 1)

# New training data
training_data_balanced = pd.concat([majority_class, minority_oversampled])
training_data_balanced = training_data_balanced.sample(frac = 1, random_state = 1).reset_index(drop = True)

# Separate balanced training data
x_train_balanced = training_data_balanced.drop(columns = ["heart_disease_status"])
y_train_balanced = training_data_balanced["heart_disease_status"]

# After oversampling
print("Balanced training set distribution:")
print(y_train_balanced.value_counts())

Balanced training set distribution:
heart_disease_status
Absent     6400
Present    6400
Name: count, dtype: int64


## Training and Comparing Models
Cross-validation will be used to compare each model and evaluate their performances on the training data.

A. Logistic Regression

In [33]:
pipe_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=1000, random_state=1))
])

scores_lr = cross_val_score(pipe_lr, x_train_balanced, y_train_balanced, cv=5, scoring="accuracy")
print(f"Logistic Regression CV Accuracy: {scores_lr.mean():.4f}")


Logistic Regression CV Accuracy: 0.5084


In [34]:
pipe_rf = Pipeline([
    ("model", RandomForestClassifier(random_state=1))
])

scores_rf = cross_val_score(pipe_rf, x_train_balanced, y_train_balanced, cv=5, scoring="accuracy")
print(f"Random Forest CV Accuracy: {scores_rf.mean():.4f}")

Random Forest CV Accuracy: 0.9798


In [35]:
pipe_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVC(probability=True, random_state=1))
])

scores_svc = cross_val_score(pipe_svc, x_train_balanced, y_train_balanced, cv=5, scoring="accuracy")
print(f"SVM CV Accuracy: {scores_svc.mean():.4f}")

SVM CV Accuracy: 0.7039


In [36]:
from sklearn.neighbors import KNeighborsClassifier

pipe_knn = Pipeline([
    ("scaler", StandardScaler()),
    ("model", KNeighborsClassifier(n_neighbors=5))
])

scores_knn = cross_val_score(pipe_knn, x_train_balanced, y_train_balanced, cv=5, scoring="accuracy")
print(f"KNN CV Accuracy: {scores_knn.mean():.4f}")

KNN CV Accuracy: 0.6945


In [40]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state=1, verbose=-1)
scores_lgbm = cross_val_score(lgbm, x_train_balanced, y_train_balanced, cv=5, scoring="accuracy")
print(f"LightGBM CV Accuracy: {scores_lgbm.mean():.4f}")

LightGBM CV Accuracy: 0.8212


In [43]:
# Create a one-hot encoded version of X
import pandas as pd

x_encoded = pd.get_dummies(x_train_balanced, drop_first=True)

# Encode target without changing original
y_temp = y_train_balanced.map({"Absent": 0, "Present": 1})

from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

xgb = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=1)

scores = cross_val_score(xgb, x_encoded, y_temp, cv=5, scoring="accuracy")
print(f"XGBoost CV Accuracy: {scores.mean():.4f}")

c:\Users\marku\OneDrive\Documents\Vscode Coding\Life-Expectancy-Project\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:29:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\marku\OneDrive\Documents\Vscode Coding\Life-Expectancy-Project\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:29:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\marku\OneDrive\Documents\Vscode Coding\Life-Expectancy-Project\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:29:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\marku\OneDrive\Documents\Vscode Coding\Life-Expectancy-Project\v

XGBoost CV Accuracy: 0.8858


✅ 1. Modeling Notebook
Purpose: Build, train, and tune machine learning models.

What to include:

Load preprocessed & split data (e.g., from a .pkl file)

Oversample training data

Train multiple models (e.g., Random Forest, Logistic Regression)

Use cross-validation

Hyperparameter tuning (GridSearchCV, RandomizedSearchCV)

Select the best model based on metrics

Make predictions on test data

Save the best model